## Linking in Spark



In [1]:
from splink.spark.jar_location import similarity_jar_location

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import types

conf = SparkConf()
# This parallelism setting is only suitable for a small toy example
conf.set("spark.driver.memory", "12g")
conf.set("spark.default.parallelism", "16")


# Add custom similarity functions, which are bundled with Splink
# documented here: https://github.com/moj-analytical-services/splink_scalaudfs
path = similarity_jar_location()
conf.set("spark.jars", path)

sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession(sc)
spark.sparkContext.setCheckpointDir("./tmp_checkpoints")

23/07/17 09:43:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import pandas as pd 
df = spark.read.csv("../../data/fake_1000.csv", header=True)

In [3]:
import splink.spark.comparison_library as cl
import splink.spark.comparison_template_library as ctl

settings = {
    "link_type": "dedupe_only",
    "comparisons": [
        ctl.name_comparison("first_name"),
        ctl.name_comparison("surname"),
        ctl.date_comparison("dob", cast_strings_to_date=True),
        cl.exact_match("city", term_frequency_adjustments=True),
        ctl.email_comparison("email", include_username_fuzzy_level=False),
    ],
    "blocking_rules_to_generate_predictions": [
        "l.first_name = r.first_name",
        "l.surname = r.surname",
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
    "em_convergence": 0.01
}

In [4]:
from splink.spark.linker import SparkLinker
linker = SparkLinker(df, settings)
deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    "l.email = r.email"
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.6)


/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/dataframe.py:169: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
--WARN-- 
 You are using datediff comparison
                        with str-casting and ANSI is not enabled. Bad dates
                        e.g. 1999-13-54 will not trigger an exception but will
                        classed as comparison level = "ELSE". Ensure date strings
                        are cleaned to remove bad dates 

/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes

In [5]:
linker.estimate_u_using_random_sampling(max_pairs=5e5)

----- Estimating u probabilities using random sampling -----
23/07/17 09:43:23 WARN DataSource: All paths were ignored:                      
  file:/Users/rosskennedy/splink/docs/demos/examples/spark/tmp_checkpoints/c7e70ae5-b182-47de-a3ee-b3826f219d1c/__splink__df_concat_with_tf_01bf6e2fe
/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use pac

In [6]:
training_blocking_rule = "l.first_name = r.first_name and l.surname = r.surname"
training_session_fname_sname = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)

training_blocking_rule = "l.dob = r.dob"
training_session_dob = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.first_name = r.first_name and l.surname = r.surname

Parameter estimates will be made for the following comparison(s):
    - dob
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname
23/07/17 09:43:33 WARN DataSource: All paths were ignored:                      
  file:/Users/rosskennedy/splink/docs/demos/examples/spark/tmp_checkpoints/c7e70ae5-b182-47de-a3ee-b3826f219d1c/__splink__df_comparison_vectors_156156e4a
/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:

In [7]:
results = linker.predict(threshold_match_probability=0.9)

23/07/17 09:34:14 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/07/17 09:34:16 WARN DataSource: All paths were ignored:                      
  file:/Users/rosskennedy/splink/docs/demos/examples/spark/tmp_checkpoints/c822bc6c-9e20-4590-91da-28bc89e52402/__splink__df_predict_937b72a4d

 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'email':
    m values not fully trained


In [8]:
results.as_pandas_dataframe(limit=5)

/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/Users/rosskennedy/Library/r-miniconda/lib/python3.9/site-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):


,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,bf_first_name,surname_l,surname_r,...,gamma_city,tf_city_l,tf_city_r,bf_city,bf_tf_adj_city,email_l,email_r,gamma_email,bf_email,match_key
0,20.775481,0.999999,172,174,Leah,Leah,4,85.028963,Russell,Russell,...,1,0.212792,0.212792,10.201219,0.259162,leahrussell@charles.net,leahrussell@charles.net,4,252.590856,0
1,22.077249,1.000000,544,546,Oliver,Oliver,4,85.028963,Jones,Jones,...,1,0.212792,0.212792,10.201219,0.259162,oliverjones82@bond.biz,oliverjones82@bond.biz,4,252.590856,0
2,13.180158,0.999892,816,819,Logan,Logan,4,85.028963,Morgan,Morgan,...,1,0.025830,0.025830,10.201219,2.134999,loganmorgan43@mcbride-king.com,loganmorgan43@icbride-kmng.com,3,212.903339,0
3,4.553689,0.959159,679,682,Elizabeth,Elizabeth,4,85.028963,Shaw,None,...,0,0.014760,0.001230,0.462959,1.000000,e.shaw@smith-hall.biz,e.shaw@smith-hall.lbiz,3,212.903339,0
4,25.224353,1.000000,683,687,Rosie,Rosie,4,85.028963,nJohnston,Johnston,...,1,0.008610,0.008610,10.201219,6.404996,rosiej32@robinson-moran.net,rosiej32@robinson-moran.net,4,252.590856,0
